In [1]:
import torch
from torch_geometric.utils import scatter

import pathpyG as pp
pp.config['torch']['device'] = 'cpu'

TODOs:
- turn into class functions: (the ones needed to estimate order)
- add as utils (computing of weigthed outdegrees and transition probabobilities)
- unit tests


In [2]:
# dag_data = pp.DAGData(pp.IndexMap(list("abcde")))
dag_data = pp.DAGData(pp.IndexMap(list("01234")))

# walk_1  =('a','b','c','d','e','c','b','a','c','d','e','c','e','d','c','a')
# walk_2  =('a','b','c','d','e','c')
# dag_data.append_walk(walk_1)
# dag_data.append_walk(walk_2)


dag_data.append_walk(list("0230230230230"), weight=2)
dag_data.append_walk(list("1241241241241"), weight=2)
# dag_data.append_walk(list("0430241"), weight=1)

m = pp.MultiOrderModel.from_DAGs(dag_data, max_order=5)

m.estimate_order(dag_data, max_order=5)

/opt/conda/lib/python3.10/site-packages/torch_geometric/edge_index.py:784: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(


2

In [3]:
from torch_geometric.loader import DataLoader
dag_graph = next(iter(DataLoader(dag_data.dags, batch_size=len(dag_data.dags)))).to(pp.config["torch"]["device"])

Why does one of those two fails while the other doesn t?

In [4]:
dag_data = pp.DAGData(pp.IndexMap(list("abcde")))

dag_data.append_walk(tuple("acd"))
dag_data.append_walk(tuple("bce"))

m = pp.MultiOrderModel.from_DAGs(dag_data, max_order=4)


def test_dof():
    line_data = pp.DAGData(pp.IndexMap(list("abcd")))
    line_data.append_walk(("a","b","c","d"))
    max_order = 4
    m = pp.MultiOrderModel.from_DAGs(line_data, max_order=max_order)
    for order in range(max_order+1):
        assert m.get_mon_dof(assumption="paths", max_order=order) == 3
    
    #########

    toy_paths_ho = pp.DAGData(pp.IndexMap(list("abcde")))
    toy_paths_ho.append_walk(("a","c","d"))
    toy_paths_ho.append_walk(("b","c","e"))
    max_order = 2
    m = pp.MultiOrderModel.from_DAGs(toy_paths_ho, max_order=max_order, mode = "propagation")
    assert m.get_mon_dof(assumption="paths", max_order=0) == 4
    assert m.get_mon_dof(assumption="paths", max_order=1) == 5
    assert m.get_mon_dof(assumption="paths", max_order=2) == 7



In [22]:
from scipy.stats import chi2
import numpy as np
def test_likelihood_ratio_test():
    significance_threshold = 0.1

    llh_zeroth = np.log(1/6) * 4 + np.log(2/6) * 2 
    llh_first = np.log(1/6) * 2 + 0 + 2 * np.log(1/2) 
    llh_second = np.log(1/6) * 2 + 0 + 0 
    dof_zeroth = 4
    dof_first = 5
    dof_second = 7
    x_01 = -2 * (llh_zeroth - llh_first)
    x_12 = -2 * (llh_first - llh_second)
    dof_diff_01 = dof_first - dof_zeroth
    dof_diff_12 = dof_second - dof_first
    p_01 = 1 - chi2.cdf(x_01, dof_diff_01)
    p_12 = 1 - chi2.cdf(x_12, dof_diff_12)

    toy_paths_ho = pp.DAGData(pp.IndexMap(list("abcde")))
    toy_paths_ho.append_walk(("a","c","d"))
    toy_paths_ho.append_walk(("b","c","e"))
    dag_graph = next(iter(DataLoader(toy_paths_ho.dags, batch_size=len(toy_paths_ho.dags)))).to(pp.config["torch"]["device"])
    max_order = 2
    m = pp.MultiOrderModel.from_DAGs(toy_paths_ho, max_order=max_order)

    bool_code_01, p_01_code = m.likelihood_ratio_test(
        dag_graph,max_order_null = 0,
        max_order = 1,
        assumption = "paths",
        significance_threshold = significance_threshold
        )

    bool_code_12, p_12_code = m.likelihood_ratio_test(
        dag_graph,max_order_null = 1,
        max_order = 2,
        assumption = "paths", 
        significance_threshold = significance_threshold
        )

    assert bool_code_01 == (p_01 < significance_threshold)
    assert np.isclose(p_01_code, p_01)
    assert bool_code_12 == (p_12 < significance_threshold)
    assert np.isclose(p_12_code, p_12)

def test_estimate_order():
    pass


test_likelihood_ratio_test()

In [9]:
import numpy as np
def test_log_likelihood():
    toy_paths_ho = pp.DAGData(pp.IndexMap(list("abcde")))
    toy_paths_ho.append_walk(("a","c","d"))
    toy_paths_ho.append_walk(("b","c","e"))
    max_order = 2
    m = pp.MultiOrderModel.from_DAGs(toy_paths_ho, max_order=max_order, mode = "propagation")
    dag_graph = next(iter(DataLoader(toy_paths_ho.dags, batch_size=len(toy_paths_ho.dags)))).to(pp.config["torch"]["device"])
    assert np.isclose(
        m.get_mon_log_likelihood(dag_graph, max_order=0),
        np.log(1/6) * 4 + np.log(2/6) * 2 
        )
    assert np.isclose(
        m.get_mon_log_likelihood(dag_graph, max_order=1),
        np.log(1/6) * 2 + 0 + 2 * np.log(1/2) 
        )
    assert np.isclose(
        m.get_mon_log_likelihood(dag_graph, max_order=2)
        ,np.log(1/6) * 2 + 0 + 0 
        )
    ################

    toy_paths = pp.DAGData(pp.IndexMap(list("abcde")))
    toy_paths.append_walk(("a","c","d"))
    toy_paths.append_walk(("b","c","e"))
    toy_paths.append_walk(("a","c","e"))
    toy_paths.append_walk(("b","c","d"))
    max_order = 2
    m = pp.MultiOrderModel.from_DAGs(toy_paths, max_order=max_order, mode = "propagation")
    dag_graph = next(iter(DataLoader(toy_paths.dags, batch_size=len(toy_paths.dags)))).to(pp.config["torch"]["device"])
    assert np.isclose(
        m.get_mon_log_likelihood(dag_graph, max_order=0), # fails already at computing log_lh here
        np.log(2/12) * 8 + np.log(4/12) * 4
        )
    assert np.isclose(
        m.get_mon_log_likelihood(dag_graph, max_order=1),
        np.log(2/12) * 4 + 0 + 4 * np.log(1/2)
        )
    assert np.isclose(
        m.get_mon_log_likelihood(dag_graph, max_order=2),
        np.log(1/6) * 4 + 0 + 4 * np.log(1/2)
        )
    
    ################

    toy_paths = pp.DAGData(pp.IndexMap(list("abcde")))
    toy_paths.append_walk(("a",))
    toy_paths.append_walk(("a","b"))
    toy_paths.append_walk(("a","b","c"))
    max_order = 2
    m = pp.MultiOrderModel.from_DAGs(toy_paths, max_order=max_order, mode = "propagation")
    dag_graph = next(iter(DataLoader(toy_paths.dags, batch_size=len(toy_paths.dags)))).to(pp.config["torch"]["device"])
    assert np.isclose(
        m.get_mon_log_likelihood(dag_graph, max_order=0), # fails already at computing log_lh here
        np.log(3/6) * 3 + np.log(2/6) * 2 + np.log(1/6) * 1
        )
    assert np.isclose(
        m.get_mon_log_likelihood(dag_graph, max_order=1),
        np.log(3/6) * 3 + 0 + 0 
        )
    assert np.isclose(
        m.get_mon_log_likelihood(dag_graph, max_order=2),
        np.log(3/6) * 3 + 0 + 0 
        )
    
test_log_likelihood()
